In [37]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
import openmeteo_requests

import requests_cache
from retry_requests import retry

from shapely import wkt
from shapely.geometry import Point
import geopandas as gpd

import warnings

warnings.filterwarnings('ignore')

## Calculate centroids for each meshblock to get coordinates

Calculate the centroid of each aggregate the centroids of the meshblocks.

By calculating the centroid, we are essentially representing the entire meshblock by a single point, which is the geometric center of the meshblock.

If we need to retrieve weather data for the entire area of the meshblock, using the centroid as a representative point is a common and practical approach I feel. 

If the area is small or the weather conditions are expected to be relatively uniform across the meshblock, this approach can be quite effective. 

I think this should be good enough to get weather information on each Meshblock. 


In [39]:
file_path = 'meshblocks-auckland.csv'
meshblock_data = pd.read_csv(file_path)

# Parse the WKT column to extract geometries
meshblock_data['geometry'] = meshblock_data['WKT'].apply(wkt.loads)

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame(meshblock_data, geometry='geometry')

# Set the initial CRS to EPSG:2193 (modify this if different)
gdf.set_crs(epsg=2193, inplace=True)

# Convert to WGS 84 (latitude and longitude)
gdf = gdf.to_crs(epsg=4326)

# Calculate centroids for each geometry
gdf['centroid'] = gdf['geometry'].centroid

# Extract latitude and longitude from centroids
gdf['latitude'] = gdf['centroid'].apply(lambda x: x.y)
gdf['longitude'] = gdf['centroid'].apply(lambda x: x.x)

# Display the data with centroids
meshblock_centroids = gdf[['MB2022_V1_00', 'latitude', 'longitude', 'SA12022_V1_00', 'SA22022_V1_00']]
meshblock_centroids.head()


,MB2022_V1_00,latitude,longitude,SA12022_V1_00,SA22022_V1_00
0,834501,-37.177258,174.756727,7010308,162400
1,4006106,-36.571222,174.446321,7001366,111600
2,150104,-36.744939,174.749479,7003017,120800
3,165300,-36.701332,174.742409,7002042,117500
4,146300,-36.528864,174.451400,7001186,110800


## Aggregate centroids by MB2022_V1_00, SA12022 and SA22022


In [40]:
# Aggregate centroids by Mesh MB2022_V1_00
mesh_centroid = meshblock_centroids.groupby('MB2022_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()
mesh_centroid.head()

# Aggregate centroids by SA12022
sa1_centroids = meshblock_centroids.groupby('SA12022_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Aggregate centroids by SA22022
sa2_centroids = meshblock_centroids.groupby('SA22022_V1_00').agg({
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

# Display aggregated centroids
mesh_centroid.head(),sa1_centroids.head(), sa2_centroids.head()


(   MB2022_V1_00   latitude   longitude
 0        146005 -36.447005  174.637796
 1        146104 -36.448109  174.671632
 2        146300 -36.528864  174.451400
 3        146400 -36.519188  174.492346
 4        146500 -36.502879  174.534495,
    SA12022_V1_00   latitude   longitude
 0        7001184 -36.464253  174.440685
 1        7001186 -36.528864  174.451400
 2        7001188 -36.483557  174.475554
 3        7001189 -36.556407  174.488362
 4        7001190 -36.519188  174.492346,
    SA22022_V1_00   latitude   longitude
 0         110700 -36.685651  174.414821
 1         110800 -36.515263  174.510876
 2         111200 -36.508434  174.652716
 3         111600 -36.610850  174.493627
 4         112100 -36.567242  174.648233)

## Get centroid co-ordinates from Meshblock, SA1, SA2 number


It depends on which level we want to use Mesh Block, SA1 or SA2

Example: find by Mesh, let say we want to find mesh 834501

In [41]:
mesh_code = 834501
mesh_coordinates = mesh_centroids[mesh_centroids['MB2022_V1_00'] == mesh_code][['latitude', 'longitude']]
mesh_coordinates 

,latitude,longitude
9494,-37.177258,174.756727


Find by SA1, Let's assume we want SA1 = 7001186

In [42]:
sa1_code = 7001186
SA1_coordinates = sa1_centroids[sa1_centroids['SA12022_V1_00'] == sa1_code][['latitude', 'longitude']]

SA1_coordinates

,latitude,longitude
1,-36.528864,174.4514


## Pass coordinates to weather code

7 days prediction

In [44]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": SA1_coordinates['latitude'], #pass SA1 latitude value here
	"longitude": SA1_coordinates['longitude'],#pass SA1 longitude value here
	"current": ["temperature_2m", "relative_humidity_2m", "is_day", "rain", "showers", "snowfall", "wind_speed_10m"],
	"hourly": ["temperature_2m", "precipitation", "rain", "showers", "snowfall", "snow_depth"],
	"daily": ["rain_sum", "showers_sum"],
	"timezone": "Pacific/Auckland"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_relative_humidity_2m = current.Variables(1).Value()
current_is_day = current.Variables(2).Value()
current_rain = current.Variables(3).Value()
current_showers = current.Variables(4).Value()
current_snowfall = current.Variables(5).Value()
current_wind_speed_10m = current.Variables(6).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")
print(f"Current relative_humidity_2m {current_relative_humidity_2m}")
print(f"Current is_day {current_is_day}")
print(f"Current rain {current_rain}")
print(f"Current showers {current_showers}")
print(f"Current snowfall {current_snowfall}")
print(f"Current wind_speed_10m {current_wind_speed_10m}")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_showers = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_rain_sum = daily.Variables(0).ValuesAsNumpy()
daily_showers_sum = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates -36.5°N 174.5°E
Elevation 30.0 m asl
Timezone b'Pacific/Auckland' b'NZST'
Timezone difference to GMT+0 43200 s
Current time 1716272100
Current temperature_2m 14.850000381469727
Current relative_humidity_2m 91.0
Current is_day 0.0
Current rain 0.0
Current showers 0.0
Current snowfall 0.0
Current wind_speed_10m 13.10419750213623
                         date  temperature_2m  precipitation  rain  showers  \
0   2024-05-20 12:00:00+00:00       16.005499            0.4   0.0      0.4   
1   2024-05-20 13:00:00+00:00       15.755500            0.0   0.0      0.0   
2   2024-05-20 14:00:00+00:00       15.955500            0.0   0.0      0.0   
3   2024-05-20 15:00:00+00:00       15.855500            0.0   0.0      0.0   
4   2024-05-20 16:00:00+00:00       15.705500            0.0   0.0      0.0   
..                        ...             ...            ...   ...      ...   
163 2024-05-27 07:00:00+00:00       12.655500            0.0   0.0      0.0   
164 2024-05-27 08:00:00+00: